In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
data= []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data.append(os.path.join(dirname, filename))
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample= pd.read_csv(data[0])
train= pd.read_csv(data[1])
test= pd.read_csv(data[2])

In [ ]:
train.head(5)

In [ ]:
train.shape

In [ ]:
train.info()


Categorical Columns - PassengerId, HomePlanet, Cabin, Destination, Name

Numerical Columns- Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck

Boolean Columns- CryoSleep, Transported, VIP

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
train['HomePlanet'].value_counts(dropna= False)
train['HomePlanet'].fillna('Earth', inplace= True)

In [ ]:
train['CryoSleep'].value_counts(dropna= False)
train['CryoSleep'].fillna(False,inplace= True)

In [ ]:
train['Cabin'].value_counts(dropna= False)

In [ ]:
def get_cabin_deck(cabin):
    try:
        return cabin.split('/')[0]
    except:
        return np.NaN
def get_cabin_side(cabin):
    try:
        return cabin.split('/')[2]
    except:
        return np.NaN

In [ ]:
train['Cabin_deck']= train['Cabin'].apply(lambda x: get_cabin_deck(x))

In [ ]:
train['Cabin_deck'].value_counts(dropna= False)

In [ ]:
train['Cabin_deck'].fillna('F',inplace= True)

In [ ]:
train['Cabin_side']= train['Cabin'].apply(lambda x: get_cabin_side(x))

In [ ]:
train['Cabin_side'].value_counts(dropna= False)

In [ ]:
np.random.seed(42)
data = np.random.choice(list(train['Cabin_side'].value_counts().index),size= 199,p= [.5,.5])

In [ ]:
missing_index=train[train["Cabin_side"].isnull()].index.tolist()

In [ ]:
fill = pd.DataFrame(index = train.index[train.Cabin_side.isnull()], data= data,columns=["Cabin_side"])
train.fillna(fill,inplace=True)
train.drop(columns=["Cabin"],inplace=True)

In [ ]:
train.info()

In [ ]:
train.isna().sum()

In [ ]:
import seaborn as sns

In [ ]:
train['Destination'].value_counts(dropna= False)

In [ ]:
train['Destination'].fillna('TRAPPIST-1e',inplace= True)

In [ ]:
train['Age'].value_counts(dropna= False)

In [ ]:
train['Age'].fillna(np.mean(train['Age']),inplace= True)

In [ ]:
train['VIP'].value_counts(dropna= False)

In [ ]:
train['VIP'].fillna(False, inplace= True)

RoomService     181

FoodCourt       183

ShoppingMall    208

Spa             183

VRDeck          188

Name            200

In [ ]:
train['RoomService'].value_counts(dropna= False)
for n in ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']:
    train[n].fillna(0,inplace= True)

In [ ]:
train['Name'].value_counts(dropna=False)

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train['VIP']= np.where(train['VIP']== True,1,0)

In [ ]:
train['CryoSleep'].value_counts()
train['CryoSleep']= np.where(train['CryoSleep']== True,1,0)

In [ ]:
train['Transported']= np.where(train['Transported']== True,1,0)

In [ ]:
train['Cabin_side'].value_counts()
train['Cabin_side']= np.where(train['Cabin_side']== 'P',1,0)

In [ ]:
from sklearn import preprocessing
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
train['Cabin_deck']= le1.fit_transform(train['Cabin_deck'])
train['HomePlanet']= le2.fit_transform(train['HomePlanet'])
train['Destination']= le3.fit_transform(train['Destination'])

In [ ]:
train['PassengerId'].head(10)

In [ ]:
plt.figure(figsize= (10,8))
sns.heatmap(train.corr(),cmap= "Blues", annot= True)
plt.show()

In [ ]:
train

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
nscaled= scaler.fit(train.loc[:, ~train.columns.isin(['Name','PassengerId'])])
ntrain= scaler.transform(train.loc[:, ~train.columns.isin(['Name','PassengerId'])])

In [ ]:
nt = pd.DataFrame(columns=["HomePlanet","CryoSleep","Destination","Age","VIP","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","Transported","Cabin_deck","Cabin_side"], data= ntrain)
nt['Transported']= train['Transported']

In [ ]:
nt['PassengerId']= train['PassengerId']
nt

In [ ]:
plt.figure(figsize= (12,10))
sns.heatmap(nt.corr(), cmap= "Blues", annot= True)
plt.show()

 0   PassengerId   8693 non-null   object
 
 1   HomePlanet    8492 non-null   object 
 
 2   CryoSleep     8476 non-null   object 
 
 3   Cabin         8494 non-null   object 
 
 4   Destination   8511 non-null   object 
 
 5   Age           8514 non-null   float64
 
 6   VIP           8490 non-null   object 
 
 7   RoomService   8512 non-null   float64
 
 8   FoodCourt     8510 non-null   float64
 
 9   ShoppingMall  8485 non-null   float64
 
 10  Spa           8510 non-null   float64
 
 11  VRDeck        8505 non-null   float64
 
 12  Name          8493 non-null   object 
 
 13  Transported   8693 non-null   bool   
 

In [ ]:
new_index= ['PassengerId','HomePlanet','CryoSleep','Cabin_deck','Cabin_side','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Name','Transported']
nt= nt.reindex(columns= new_index)
nt.head(5)

In [ ]:
nt = nt.drop('Name',axis=1)
nt.head(5)

In [ ]:
def get_passenger_id(pid): return pid.split('_')[0]
def get_passenger_grp(pid): return pid.split('_')[1] 

In [ ]:
nt['PassengerUid'] = nt['PassengerId'].apply(lambda x: get_passenger_id(x))
nt['PassengerGroup'] = nt['PassengerId'].apply(lambda x: get_passenger_grp(x))

In [ ]:
PassengerUid= nt.pop('PassengerUid')

In [ ]:
PassengerUid

In [ ]:
nt = nt.drop(['PassengerId'], axis= 1)

In [ ]:
new_index= ['PassengerGroup','HomePlanet','CryoSleep','Cabin_deck','Cabin_side','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Transported']
nt= nt.reindex(columns= new_index)
nt['Transported'] = train['Transported']

In [ ]:
nt.head(5)

In [ ]:
nt['PassengerGroup'] = pd.to_numeric(nt['PassengerGroup'])

In [ ]:
nt.info()

In [ ]:
plt.figure(figsize= (12,10))
sns.heatmap(nt.corr(), cmap= "Blues", annot= True)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X= nt.drop('Transported',axis= 1)
y= nt.pop('Transported')

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# performance measurement
def getpref(y_pred): 
    print(f'Confusion Matrix :\n {confusion_matrix(y_test, y_pred)}')
    print(f'Accuracy         : {round(accuracy_score(y_test, y_pred),2)}')
    print(f'F1 Score         : {round(f1_score(y_test, y_pred),2)}')
    print(f'Precision        : {round(precision_score(y_test, y_pred),2)}')
    print(f'Recall           : {round(recall_score(y_test, y_pred),2)}')

In [ ]:
#Logistic Regression

lr= LogisticRegression()
lr.fit(X_train, y_train)
lr_pred= lr.predict(X_test)
getpref(lr_pred)

In [ ]:
# Decision Tree Classifier

dtc = DecisionTreeClassifier(criterion= "entropy", splitter= "best", min_impurity_decrease=0.01,min_samples_split=10,min_samples_leaf=10)
dtc.fit(X_train, y_train)
dtc_pred= dtc.predict(X_test)
getpref(dtc_pred)

In [ ]:
# Random Forest Classifier

rfc = RandomForestClassifier(n_estimators= 30,n_jobs= -1,criterion= "entropy", min_impurity_decrease=0.01,min_samples_split=10,min_samples_leaf=10)
rfc.fit(X_train, y_train)
rfc_pred= rfc.predict(X_test)
getpref(rfc_pred)

In [ ]:
# Extra Trees Classifier

etc = ExtraTreesClassifier(n_estimators= 30,n_jobs= -1,criterion= "gini", min_impurity_decrease=0.01,min_samples_split=5,min_samples_leaf=5)
etc.fit(X_train, y_train)
etc_pred= etc.predict(X_test)
getpref(etc_pred)

SUBMISSION


In [ ]:
# lets Check if there are missing values
print("Missing Value in Each Variable\n")
for feature in test.columns:
  print(f"{feature} : {test[feature].isna().sum()}")

In [ ]:
test

In [ ]:
test['PassengerGroup'] = test['PassengerId'].apply(lambda x: x.split('_')[1])
test['PassengerUid'] = test['PassengerId'].apply(lambda x: x.split('_')[0])


In [ ]:
test.pop('PassengerId')

In [ ]:
test

In [ ]:
test['HomePlanet'].fillna('Earth', inplace= True)
test['CryoSleep'].fillna(False, inplace= True)
test['Destination'].fillna('TRAPPIST-1e', inplace= True)
test['Cabin_deck'] = test['Cabin'].apply(lambda x: get_cabin_deck(x))
test['Cabin_side'] = test['Cabin'].apply(lambda x: get_cabin_side(x))
test['Age'].fillna(np.mean(test['Age']), inplace= True)
test['VIP'].fillna(False, inplace= True)
test = test.drop('Name', axis= 1)

for n in ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]:
    test[n].fillna(0, inplace= True)

test['Cabin_side'] = np.where(test['Cabin_side'] == 'P',1,0)
test['Cabin_deck']= le1.fit_transform(test['Cabin_deck'])
test['HomePlanet']= le2.fit_transform(test['HomePlanet'])
test['Destination']= le3.fit_transform(test['Destination'])

test = test.drop('Cabin', axis= 1)

In [ ]:
test

In [ ]:
newpUid = test['PassengerUid']
newpGroup= test['PassengerGroup']
sclr = MinMaxScaler()
nscaled= sclr.fit(test.loc[:, ~test.columns.isin(['PassengerGroup'])])
ntest= sclr.transform(test.loc[:, ~test.columns.isin(['PassengerGroup'])])

ntest

In [ ]:
test = pd.DataFrame(columns=["HomePlanet","CryoSleep","Destination","Age","VIP","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","PassengerUid","Cabin_deck","Cabin_side"], data= ntest)

In [ ]:
test

In [ ]:
test = test.drop("PassengerUid", axis =1)
test['PassengerGroup'] = newpGroup

In [ ]:
test

PassengerGroup	HomePlanet	CryoSleep	Cabin_deck	Cabin_side	Destination	Age	VIP	RoomService	FoodCourt	ShoppingMall	Spa	VRDeck	Transported

In [ ]:
new_index= ['PassengerGroup','HomePlanet','CryoSleep','Cabin_deck','Cabin_side','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
test= test.reindex(columns= new_index)

In [ ]:
test

In [ ]:
y_test_pred = dtc.predict(test)

In [ ]:
submit=pd.DataFrame(data=y_test_pred,columns=["Transported"])
submit_test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submit["PassengerId"]=submit_test["PassengerId"]
submit["Transported"].replace({1:True,0:False},inplace=True)

In [ ]:
submit.to_csv("submission.csv",index=False)